In [1]:
from CONSTANT import *
from tools import *
import pandas as pd
import numpy as np
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoderLayer, TransformerDecoder

from models import SigRep

import torch

from torch import nn
import torch.nn.functional as F
from tqdm import tqdm


use_cuda = True
valid='loso'

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('using', device)

from representation.SigRepre import MultiSignalRepresentation

using cuda


In [2]:
class Params(object):
    def __init__(self, lr=0.001, epoch=100, valid='loso', target='valence', batch_size=64, use_cuda=use_cuda):
        self.batch_size = batch_size
        self.valid = valid
        self.target = target
        self.epoch = epoch
        self.lr = lr
        self.metrics_dict = {}
        self.debug = True
        self.out_channels = 32
        self.out_channels = 32
        self.hidden_size = 64
        self.num_layers = 1
        self.fcn_input = 50432
        self.dropout=0.2
        self.use_cuda=use_cuda
        self.device=torch.device(
            'cuda' if torch.cuda.is_available() and self.use_cuda else 'cpu')

In [3]:
spliter = load_model(r'./processed_signal/HKU956/400_4s_step_2s_spliter.pkl')
data = pd.read_pickle(r'./processed_signal/HKU956/400_4s_step_2s.pkl')
# data = pd.read_csv(r'./processed_signal/HKU956/400_4s_step_2s.csv')

for k in spliter[valid]:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [4]:
args = Params()
dataprepare = DataPrepare(args=args, target='valence', data=data, train_index=train_index, test_index=test_index, device=device, batch_size=4)
# train_dataloader, test_dataloader = dataprepare.get_data()

(1000, 4, 400) (1000, 1) (100, 4, 400) (100, 1)


In [6]:
# xtest = dataprepare.xtest
# xtrain = dataprepare.xtrain
# ytest = dataprepare.ytest
# ytrain = dataprepare.ytrain
# xtest.shape, xtrain.shape, ytest.shape, ytrain.shape

(torch.Size([100, 4, 400]),
 torch.Size([1000, 4, 400]),
 torch.Size([100, 1]),
 torch.Size([1000, 1]))

In [5]:
model = MultiSignalRepresentation(output_size=40, device=args.device)
model = model.to(args.device)
model.load_state_dict(torch.load(r'./representation/mask08ep13.pt'))

<All keys matched successfully>

In [6]:
train_data, test_data = dataprepare.get_data()
encoded_signals = []

for i, (x, y) in tqdm(enumerate(train_data)):
    encodedx = model.encoder(x).flatten(start_dim=1)
    encoded_signals.append(encodedx)

19it [00:14,  1.35it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 5.12 GiB already allocated; 0 bytes free; 5.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
encoded_signals = np.concatenate(encoded_signals)
encoded_signals.shape

In [ ]:
# encoded_xtrain = model.encoder(xtrain).flatten(start_dim=1)
# encoded_xtrain.shape